# Data Vault Relationship Checks (Silver Layer)

Validate hub/link/satellite relationships and look for orphaned keys in the silver layer.

In [2]:
from pathlib import Path
import sys

repo_root = Path.cwd().resolve()
if (repo_root / 'backend').is_dir():
    pass
elif (repo_root.parent / 'backend').is_dir():
    repo_root = repo_root.parent
else:
    raise RuntimeError('Run this notebook from the repo or backend folder.')

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from backend.db import session


RuntimeError: Run this notebook from the repo or backend folder.

In [ ]:
print(f'Database path: {session.DB_PATH}')
tables = session.get_tables()
print(f'Found {len(tables)} tables.')


In [ ]:
def has_table(name: str) -> bool:
    return name in tables

def run_if_exists(name: str, query: str):
    if not has_table(name):
        print(f'Skipping {name}: table not found.')
        return None
    result = session.execute_query(query)
    print(name, result[0])
    return result


## Hub Row Counts

In [ ]:
run_if_exists('hub_resort', 'SELECT COUNT(*) AS rows FROM hub_resort')
run_if_exists('hub_station', 'SELECT COUNT(*) AS rows FROM hub_station')
run_if_exists('hub_zone', 'SELECT COUNT(*) AS rows FROM hub_zone')
run_if_exists('hub_office', 'SELECT COUNT(*) AS rows FROM hub_office')


## Link Orphans (Keys Missing in Hubs)

In [ ]:
run_if_exists(
    'link_resort_zone',
    '''
    SELECT
      COUNT(*) AS link_rows,
      SUM(CASE WHEN r.resort_key IS NULL THEN 1 ELSE 0 END) AS missing_resort_keys,
      SUM(CASE WHEN z.zone_key IS NULL THEN 1 ELSE 0 END) AS missing_zone_keys
    FROM link_resort_zone l
    LEFT JOIN hub_resort r ON l.resort_key = r.resort_key
    LEFT JOIN hub_zone z ON l.zone_key = z.zone_key
    '''
)

run_if_exists(
    'link_resort_station',
    '''
    SELECT
      COUNT(*) AS link_rows,
      SUM(CASE WHEN r.resort_key IS NULL THEN 1 ELSE 0 END) AS missing_resort_keys,
      SUM(CASE WHEN s.station_key IS NULL THEN 1 ELSE 0 END) AS missing_station_keys
    FROM link_resort_station l
    LEFT JOIN hub_resort r ON l.resort_key = r.resort_key
    LEFT JOIN hub_station s ON l.station_key = s.station_key
    '''
)

run_if_exists(
    'link_zone_office',
    '''
    SELECT
      COUNT(*) AS link_rows,
      SUM(CASE WHEN z.zone_key IS NULL THEN 1 ELSE 0 END) AS missing_zone_keys,
      SUM(CASE WHEN o.office_key IS NULL THEN 1 ELSE 0 END) AS missing_office_keys
    FROM link_zone_office l
    LEFT JOIN hub_zone z ON l.zone_key = z.zone_key
    LEFT JOIN hub_office o ON l.office_key = o.office_key
    '''
)


## Satellite Orphans (Keys Missing in Hubs)

In [ ]:
run_if_exists(
    'sat_resort_details',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN r.resort_key IS NULL THEN 1 ELSE 0 END) AS missing_resort_keys
    FROM sat_resort_details s
    LEFT JOIN hub_resort r ON s.resort_key = r.resort_key
    '''
)

run_if_exists(
    'sat_station_details',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.station_key IS NULL THEN 1 ELSE 0 END) AS missing_station_keys
    FROM sat_station_details s
    LEFT JOIN hub_station h ON s.station_key = h.station_key
    '''
)

run_if_exists(
    'sat_observation',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.station_key IS NULL THEN 1 ELSE 0 END) AS missing_station_keys
    FROM sat_observation s
    LEFT JOIN hub_station h ON s.station_key = h.station_key
    '''
)

run_if_exists(
    'sat_zone_details',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.zone_key IS NULL THEN 1 ELSE 0 END) AS missing_zone_keys
    FROM sat_zone_details s
    LEFT JOIN hub_zone h ON s.zone_key = h.zone_key
    '''
)

run_if_exists(
    'sat_office_details',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.office_key IS NULL THEN 1 ELSE 0 END) AS missing_office_keys
    FROM sat_office_details s
    LEFT JOIN hub_office h ON s.office_key = h.office_key
    '''
)

run_if_exists(
    'sat_forecast_period',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.resort_key IS NULL THEN 1 ELSE 0 END) AS missing_resort_keys
    FROM sat_forecast_period s
    LEFT JOIN hub_resort h ON s.resort_key = h.resort_key
    '''
)

run_if_exists(
    'sat_forecast_hourly',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.resort_key IS NULL THEN 1 ELSE 0 END) AS missing_resort_keys
    FROM sat_forecast_hourly s
    LEFT JOIN hub_resort h ON s.resort_key = h.resort_key
    '''
)

run_if_exists(
    'sat_grid_data',
    '''
    SELECT
      COUNT(*) AS rows,
      SUM(CASE WHEN h.resort_key IS NULL THEN 1 ELSE 0 END) AS missing_resort_keys
    FROM sat_grid_data s
    LEFT JOIN hub_resort h ON s.resort_key = h.resort_key
    '''
)


## Link Coverage Snapshots

In [ ]:
run_if_exists(
    'link_resort_station',
    '''
    SELECT
      resort_key,
      COUNT(*) AS station_links
    FROM link_resort_station
    GROUP BY resort_key
    ORDER BY station_links DESC
    LIMIT 10
    '''
)

run_if_exists(
    'link_resort_zone',
    '''
    SELECT
      resort_key,
      COUNT(*) AS zone_links
    FROM link_resort_zone
    GROUP BY resort_key
    ORDER BY zone_links DESC
    LIMIT 10
    '''
)
